GPS(General Problem Solver)
 
 The original GPS was implemented in a now-arcane language called Information Processing Language (IPL)

In [1]:
import copy

class GPS:
    def __init__(self, initial_state, goal_state, operators):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.operators = operators

    def solve(self):
        """
        The main means-ends analysis algorithm.
        """
        # A list to keep track of the sequence of applied operators
        plan = []
        current_state = self.initial_state
        
        # Continue until the current state satisfies the goal
        while not self.is_goal_achieved(current_state, self.goal_state):
            # Find the differences between the current state and the goal state
            diffs = self.find_differences(current_state, self.goal_state)
            
            if not diffs:
                print("No differences found, but goal not achieved. Cannot solve.")
                return None

            # For simplicity, we'll work on the first detected difference
            diff = diffs[0]
            
            # Find a relevant operator to reduce this difference
            op = self.find_relevant_operator(diff)

            if not op:
                print(f"No operator found to solve difference: {diff}")
                return None
            
            # If the operator's preconditions are not met, set a subgoal
            if not self.are_preconditions_met(current_state, op.preconditions):
                subgoal_plan = self.achieve_subgoal(current_state, op.preconditions)
                if subgoal_plan is None:
                    print("Failed to achieve subgoal.")
                    return None
                
                # Apply the subgoal plan to the current state
                for sub_op in subgoal_plan:
                    current_state = self.apply_operator(current_state, sub_op)
                    plan.append(sub_op)

            # Now that preconditions are met, apply the main operator
            current_state = self.apply_operator(current_state, op)
            plan.append(op)
            
        return plan

    def is_goal_achieved(self, state, goal):
        return all(item in state for item in goal)

    def find_differences(self, current_state, goal_state):
        return [item for item in goal_state if item not in current_state]

    def find_relevant_operator(self, diff):
        for op in self.operators:
            if diff in op.add_list:
                return op
        return None

    def are_preconditions_met(self, state, preconditions):
        return all(item in state for item in preconditions)

    def apply_operator(self, state, op):
        new_state = set(state)
        for item in op.del_list:
            if item in new_state:
                new_state.remove(item)
        for item in op.add_list:
            new_state.add(item)
        return frozenset(new_state)

    def achieve_subgoal(self, state, subgoal):
        """
        A recursive call to solve for the preconditions of an operator.
        """
        gps_sub = GPS(state, subgoal, self.operators)
        return gps_sub.solve()

class Operator:
    def __init__(self, action, preconditions, add_list, del_list):
        self.action = action
        self.preconditions = preconditions
        self.add_list = add_list
        self.del_list = del_list

In [6]:
# Define the problem states and operators
initial_state = frozenset(["at door", "on floor", "box at window", "hungry"])
goal_state = frozenset(["has bananas"])

# Define the operators for the monkey and bananas problem
operators = [
    Operator(
        action="push box to under bananas",
        preconditions=["at door", "on floor"],
        add_list=["box under bananas", "at box"],
        del_list=["box at window", "at door"],
    ),
    Operator(
        action="climb on box",
        preconditions=["at box", "on floor"],
        add_list=["on box"],
        del_list=["on floor", "at box"],
    ),
    Operator(
        action="grasp bananas",
        preconditions=["on box", "box under bananas"],
        add_list=["has bananas"],
        del_list=["hungry"],
    ),
    Operator(
        action="walk to door",
        preconditions=["on floor", "at box"],
        add_list=["at door"],
        del_list=["at box"],
    ),
]

# Create a GPS instance and solve the problem
gps_solver = GPS(initial_state, goal_state, operators)
solution_plan = gps_solver.solve()

# Print the solution
if solution_plan:
    print("\n--- Solution Found ---")
    for i, op in enumerate(solution_plan):
        print(f"Step {i+1}: {op.action}")
else:
    print("\n--- No Solution Found ---")

---
ATTEMPTING GOAL: frozenset({'has bananas'})
CURRENT STATE:   frozenset({'at door', 'on floor', 'box at window', 'hungry'})
DIFFERENCES:     ['has bananas']
-> For difference 'has bananas', selecting operator: 'grasp bananas'
   Preconditions for 'grasp bananas' are: ['on box', 'box under bananas']
   ==> SETTING NEW SUBGOAL to achieve these preconditions.
  ---
  ATTEMPTING GOAL: ['on box', 'box under bananas']
  CURRENT STATE:   frozenset({'at door', 'on floor', 'box at window', 'hungry'})
  DIFFERENCES:     ['on box', 'box under bananas']
  -> For difference 'on box', selecting operator: 'climb on box'
     Preconditions for 'climb on box' are: ['at box', 'on floor']
     ==> SETTING NEW SUBGOAL to achieve these preconditions.
    ---
    ATTEMPTING GOAL: ['at box', 'on floor']
    CURRENT STATE:   frozenset({'at door', 'on floor', 'box at window', 'hungry'})
    DIFFERENCES:     ['at box']
    -> For difference 'at box', selecting operator: 'push box to under bananas'
       Pre

In [4]:
import copy

class GPS:
    def __init__(self, initial_state, goal_state, operators):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.operators = operators
        self.plan = []

    def solve(self):
        """
        A wrapper for the main means-ends analysis algorithm.
        """
        if self.achieve(self.goal_state):
            return self.plan
        else:
            return None

    def achieve(self, goals):
        """
        The core means-ends analysis logic. Tries to achieve a set of goals.
        """
        current_state = self.get_current_state()

        # Check if goals are already met
        if all(g in current_state for g in goals):
            return True

        # Find differences (unachieved goals)
        diffs = [g for g in goals if g not in current_state]

        for diff in diffs:
            # Find a relevant operator to resolve the difference
            op = self.find_relevant_operator(diff)
            if not op:
                continue # Try the next difference if no operator found

            # If the operator can be applied, great. But first, we must satisfy its preconditions.
            # This is the recursive, subgoal-setting part of GPS.
            if self.achieve(op.preconditions):
                print(f"Applying operator: {op.action}")
                # Add the operator to the final plan
                self.plan.append(op)
                return self.achieve(goals) # Re-attempt to achieve the original goals

        return False

    def find_relevant_operator(self, diff):
        # Find an operator that adds the desired state (the 'diff')
        for op in self.operators:
            if diff in op.add_list:
                return op
        return None

    def get_current_state(self):
        # The current state is the initial state plus all the effects of the operators applied so far
        state = set(self.initial_state)
        for op in self.plan:
            for item in op.del_list:
                if item in state:
                    state.remove(item)
            for item in op.add_list:
                state.add(item)
        return frozenset(state)


class Operator:
    def __init__(self, action, preconditions, add_list, del_list):
        self.action = action
        self.preconditions = preconditions
        self.add_list = add_list
        self.del_list = del_list

    def __repr__(self):
        return self.action

# --- Problem Definition ---

# Define the initial state of the world
initial_state = frozenset(["at door", "on floor", "box at window", "hungry"])

# Define the goal we want to achieve
goal_state = frozenset(["has bananas"])

# Define the operators (actions) the monkey can perform
operators = [
    Operator(
        action="push box to under bananas",
        preconditions=["at door", "on floor", "box at window"],
        add_list=["box under bananas", "at box"],
        del_list=["box at window", "at door"],
    ),
    Operator(
        action="climb on box",
        preconditions=["at box", "on floor"], # Corrected preconditions
        add_list=["on box"],
        del_list=["on floor", "at box"],
    ),
    Operator(
        action="grasp bananas",
        preconditions=["on box", "box under bananas"],
        add_list=["has bananas"],
        del_list=["hungry"],
    )
]

# --- Solve the Problem ---

# Create a GPS instance and solve
gps_solver = GPS(initial_state, goal_state, operators)
solution_plan = gps_solver.solve()

# Print the final plan
if solution_plan:
    print("\n--- Solution Found ---")
    for i, op in enumerate(solution_plan):
        print(f"Step {i+1}: {op.action}")
else:
    print("\n--- No Solution Found ---")

Applying operator: push box to under bananas
Applying operator: climb on box
Applying operator: grasp bananas

--- Solution Found ---
Step 1: push box to under bananas
Step 2: climb on box
Step 3: grasp bananas


In [5]:
class GPS:
    def __init__(self, initial_state, goal_state, operators):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.operators = operators
        self.plan = []

    def solve(self):
        """A wrapper for the main means-ends analysis algorithm."""
        # We start the recursive process with the main goal, at depth 0.
        if self.achieve(self.goal_state, depth=0):
            return self.plan
        else:
            return None

    def achieve(self, goals, depth):
        """The core means-ends analysis logic, now with tracing."""
        indent = "  " * depth  # Indentation for visual hierarchy
        print(f"{indent}---")
        print(f"{indent}ATTEMPTING GOAL: {goals}")
        
        current_state = self.get_current_state()
        print(f"{indent}CURRENT STATE:   {current_state}")

        # Check if goals are already met in the current state
        if all(g in current_state for g in goals):
            print(f"{indent}SUCCESS: Goal {goals} already achieved.")
            return True

        # Find differences between the current state and the desired goals
        diffs = [g for g in goals if g not in current_state]
        print(f"{indent}DIFFERENCES:     {diffs}")
        
        # Try to resolve each difference
        for diff in diffs:
            # Find a relevant operator that can add the missing state ('diff')
            op = self.find_relevant_operator(diff)
            if not op:
                continue

            print(f"{indent}-> For difference '{diff}', selecting operator: '{op.action}'")

            # HERE IS THE KEY: The operator's preconditions become the new subgoal.
            # We must achieve them before we can apply the operator.
            print(f"{indent}   Preconditions for '{op.action}' are: {op.preconditions}")
            print(f"{indent}   ==> SETTING NEW SUBGOAL to achieve these preconditions.")

            # Recursive call: try to achieve the preconditions.
            # Notice we increase the depth for the print statements.
            if self.achieve(op.preconditions, depth + 1):
                # If the subgoal was successful, we can now apply the operator
                print(f"{indent}*** SUBGOAL MET! Applying operator '{op.action}' ***")
                self.plan.append(op)
                
                # After applying the operator, we must re-evaluate the original goal.
                # This ensures all parts of the original goal are met.
                return self.achieve(goals, depth)

        # If we loop through all differences and can't find a way to solve them
        print(f"{indent}FAILED: Could not achieve goal {goals}")
        return False

    def find_relevant_operator(self, diff):
        for op in self.operators:
            if diff in op.add_list:
                return op
        return None

    def get_current_state(self):
        state = set(self.initial_state)
        for op in self.plan:
            for item in op.del_list:
                if item in state:
                    state.remove(item)
            for item in op.add_list:
                state.add(item)
        return frozenset(state)